In [ ]:
import numpy as np
import random
import pickle
import pymysql
import pymysql.cursors
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
from extractor_accelerated import *

In [ ]:
f = open("final_model.pickle", 'rb')
clf = pickle.load(f)
f.close()

In [ ]:
conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='root',db='gingko')
cursor = conn.cursor()
cursor.execute('select count(*) from web_pages2')
values = cursor.fetchall()
print(values[0][0])

In [ ]:
cursor.execute('select site, html from web_pages2')

In [ ]:
scores = dict()
counts = dict()
_mean = np.array([35.9133987, 3.4475999, 67.1036072, 0.6936892, 0.6221208, 0.9910553])
_std = np.array([96.6447144, 14.6807480, 138.3404999, 0.8288413, 0.3573704, 0.0016996])

In [ ]:
page = cursor.fetchone()
for p in tqdm(range(130000)):
    soup = BeautifulSoup(page[1], 'html.parser')
    vector = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
    vector[0,0] = get_img_number(soup)
    vector[0,1] = get_ads_number(soup)
    vector[0,2] = get_reading_level(page[1])
    vector[0,3] = social_media_score(soup)
    vector[0,4] = citation_score(soup)
    vector[0,5] = sentiment_analysis(page[1])
    vector = (vector - _mean) / _std
    curr_score = clf.predict_proba(vector)[0,0]
    curr_site = page[0]
    if curr_site not in counts:
        counts[curr_site] = 1
        scores[curr_site] = curr_score
    else:
        counts[curr_site] += 1
        scores[curr_site] = (scores[curr_site]*(counts[curr_site]-1) + curr_score) / counts[curr_site]
    #
    page = cursor.fetchone()

In [ ]:
outf = open("scores.pickle", 'wb')
pickle.dump(scores, outf)
outf.close()